## 5.1 영업 성공 예측(분류)

### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

In [ ]:
# 혼동행렬 출력용 함수

def make_cm(matrix, columns):
    # matrix : 넘파이 배열
    # columns : 필드명 리스트
    n = len(columns)
    
    # '정답 데이터'를 n번 반복해 연접한 리스트
    act = ['정답데이터'] * n
    pred = ['예측결과'] * n
    
    # 데이터프레임 생성
    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

### 5.1.4 데이터 읽어 들이기부터 데이터 확인까지

#### 데이터 읽어 들이기

In [ ]:
# 데이터 집합을 내려받아 압축 해제
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O bank.zip | tail -n 1
!unzip -o bank.zip | tail -n 1

# 역주: 위 명령에서 오류가 날 경우 URL의 파일을 직접 내려받아 notebooks 디렉토리에
# 압축을 해제하면 정상 진행할 수 있습니다.

# bank-full.csv 파일을 데이터프레임으로 읽어 들이기
df_all = pd.read_csv('bank-full.csv', sep=';')

# 우리말 필드명을 정의
columns = [
    '연령', '직업', '혼인_여부', '학력', '채무불이행', '평균잔고', 
    '주택대출', '신용대출', '연락수단', '마지막통화일', 
    '마지막통화월', '마지막통화시간', '통화횟수_캠페인중', 
    '마지막영업후_경과일수', '통화횟수_캠페인전', '지난영업_결과', 
    '이번영업_결과' 
]
df_all.columns = columns

#### 데이터 확인

In [ ]:
# 데이터프레임 내용 확인
display(df_all.head())

In [ ]:
# 데이터 건수와 필드 수 확인
print(df_all.shape)
print()

# '이번영업_결과' 필드의 값 분포 확인
print(df_all['이번영업_결과'].value_counts())
print()

# 영업 성공률 
rate = df_all['이번영업_결과'].value_counts()['yes']/len(df_all)
print(f'영업 성공률: {rate:.4f}')

In [ ]:
# 누락값 확인
print(df_all.isnull().sum())

### 5.1.5 데이터 전처리 및 데이터 분할

#### 데이터 전처리

##### 전처리 1단계

In [ ]:
# get_dummies 함수를 사용해 범주 값에 원-핫 인코딩 적용

# 필드에 원-핫 인코딩을 적용하는 함수
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df.drop([column],axis=1),df_dummy],axis=1)
    return df

df_all2 = df_all.copy()
df_all2 = enc(df_all2, '직업')
df_all2 = enc(df_all2, '혼인_여부')
df_all2 = enc(df_all2, '학력')
df_all2 = enc(df_all2, '연락수단')
df_all2 = enc(df_all2, '지난영업_결과')

# 결과 확인
display(df_all2.head())

##### 전처리 2단계

In [ ]:
# yes/no를 1과 0으로 변환

# 이진 레이블값(yes/no)를 정수(1/0)으로 변환하는 함수
def enc_bin(df, column):
    df[column] = df[column].map(dict(yes=1, no=0))
    return df

df_all2 = enc_bin(df_all2, '채무불이행')
df_all2 = enc_bin(df_all2, '주택대출')
df_all2 = enc_bin(df_all2, '신용대출')
df_all2 = enc_bin(df_all2, '이번영업_결과')

# 결과 확인
display(df_all2.head())

##### 전처리 3단계

In [ ]:
# 달 이름(jan, feb ..)을 숫자(1, 2 ..)로 변환

month_dict = dict(jan=1, feb=2, mar=3, apr=4,
    may=5, jun=6, jul=7, aug=8, sep=9, oct=10,
    nov=11, dec=12)

def enc_month(df, column):
    df[column] = df[column].map(month_dict)
    return df

df_all2 = enc_month(df_all2, '마지막통화월')

# 결과 확인
display(df_all2.head())

#### 데이터 분할

In [ ]:
# 입력 데이터와 정답 데이터를 나누기
x = df_all2.drop('이번영업_결과', axis=1)
y = df_all2['이번영업_결과'].values

# 학습 데이터와 검증 데이터를 나누기
# 학습 데이터 60%, 검증 데이터 40%의 비율이 되도록 분할
test_size = 0.4

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
  x, y, test_size=test_size, random_state=random_seed,
  stratify=y)

### 5.1.6 알고리즘 선택하기

#### 알고리즘 선택

In [ ]:
# 후보 알고리즘 리스트 만들기

# 로지스틱 회귀 (4.3.3)
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# 결정 트리 (4.3.6)
from sklearn.tree import DecisionTreeClassifier
algorithm2 = DecisionTreeClassifier(random_state=random_seed)

# 랜덤 포레스트 (4.3.7)
from sklearn.ensemble import RandomForestClassifier
algorithm3 = RandomForestClassifier(random_state=random_seed)

# XGBoost (4.3.8)
from xgboost import XGBClassifier
algorithm4 = XGBClassifier(random_state=random_seed)

algorithms = [algorithm1, algorithm2, algorithm3, algorithm4]

In [ ]:
# 교차검증법을 적용해 최적의 알고리즘을 선정한다
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
    # 교차검증법 적용
    scores = cross_val_score(algorithm , x_train, y_train,
        cv=stratifiedkfold, scoring='roc_auc')
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'평균 정확도: {score:.4f}  개별 정확도: {scores}  {name}')

#### 결론
XGBoost가 네 가지 알고리즘 중 가장 높은 성능을 보였다.  
-> 지금부터는 XGBoost를 사용한다.

### 5.1.7 학습, 예측, 평가 단계

In [ ]:
# 알고리즘 선택 (XGBoost)
algorithm = XGBClassifier(random_state=random_seed)

# 학습
algorithm.fit(x_train, y_train)

# 예측
y_pred = algorithm.predict(x_test)

In [ ]:
# 평가

# 혼동행렬 출력
from sklearn.metrics import confusion_matrix
df_matrix = make_cm(
    confusion_matrix(y_test, y_pred), ['실패', '성공'])
display(df_matrix)

# 정확률, 재현율, F-점수 계산하기
from sklearn.metrics import precision_recall_fscore_support
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test, y_pred, average='binary')
print(f'정밀도: {precision:.4f}  재현율: {recall:.4f}  F-점수: {fscore:.4f}')

### 5.1.8 튜닝

#### 확률값의 도수분포 그래프

In [ ]:
# 확률값의 도수분포 그래프
import seaborn as sns

# y=0인 데이터의 확률값 구하기
y_proba0 = algorithm.predict_proba(x_test)[:,1]

# y_test=0과 y_test=1로 데이터를 분할
y0 = y_proba0[y_test==0]
y1 = y_proba0[y_test==1]

# 산포도 그리기
plt.figure(figsize=(6,6))
plt.title('확률값의 도수분포')
sns.distplot(y1, kde=False, norm_hist=True,
    bins=50, color='b', label='성공')
sns.distplot(y0, kde=False, norm_hist=True,
    bins=50, color='k', label='실패')
plt.xlabel('확률값')
plt.legend()
plt.show()

#### predict_proba 함수를 사용해 0.5 외의 값을 역치로 설정해 예측을 수행
(4.4절 참조)  

In [ ]:
# 설정한 역치 값에 대해 예측을 수행하는 함수
def pred(algorithm, x, thres):
    # 확률값 꺼내기 (행렬)
    y_proba = algorithm.predict_proba(x)

    # 예측결과 1의 함숫값
    y_proba1 =  y_proba[:,1]

    # 예측결과 1의 함숫값이 역치보다 큰가? 
    y_pred = (y_proba1 > thres).astype(int)
    return y_pred

In [ ]:
# 역치를 0.05씩 감소시켜가며 정확률, 재현율, F-점수를 계산한다
thres_list = np.arange(0.5, 0, -0.05)

for thres in thres_list:
    y_pred = pred(algorithm, x_test, thres)
    pred_sum =  y_pred.sum()
    precision, recall, fscore, _ = precision_recall_fscore_support(
        y_test, y_pred, average='binary')
    print(f'역치: {thres:.2f} 양성 예측 수: {pred_sum}\
 정밀도: {precision:.4f} 재현율: {recall:.4f}  F-점수: {fscore:.4f})')

In [ ]:
# F-점수가 최대가 되는 역치는 0.30
y_final = pred(algorithm, x_test, 0.30)

# 혼동행렬을 출력
df_matrix2 = make_cm(
    confusion_matrix(y_test, y_final), ['실패', '성공'])
display(df_matrix2)

# 정확률, 재현율, F-점수를 계산
precision, recall, fscore, _ = precision_recall_fscore_support(
    y_test, y_final, average='binary')
print(f'정밀도: {precision:.4f}  재현율: {recall:.4f}\
  F-점수: {fscore:.4f}')

### 5.1.9 중요도 분석

In [ ]:
# 중요도 분석

# 중요도 벡터 계산
importances = algorithm.feature_importances_

# 필드명을 키로 Series 객체를 생성
w = pd.Series(importances, index=x.columns)

# 내림차순으로 정렬
u = w.sort_values(ascending=False)

# 상위 1개 항목을 추출
v = u[:10]

# 중요도의 막대그래프를 출력
plt.title('입력 필드의 중요도')
plt.bar(range(len(v)), v, color='b', align='center')
plt.xticks(range(len(v)), v.index, rotation=90)
plt.show()

In [ ]:
column = '지난영업_결과_success'

sns.distplot(x_test[y_test==1][column], kde=False, norm_hist=True,
            bins=5,color='b', label='성공')
sns.distplot(x_test[y_test==0][column], kde=False, norm_hist=True,
             bins=5,color='k', label='실패')

plt.legend()
plt.show()

In [ ]:
column = '마지막통화시간'

sns.distplot(x_test[y_test==1][column], kde=False, norm_hist=True,
             bins=50, color='b', label='성공')
sns.distplot(x_test[y_test==0][column], kde=False, norm_hist=True,
             bins=50, color='k', label='실패')

plt.legend()
plt.show()

In [ ]:
column = '연락수단_unknown'

sns.distplot(x_test[y_test==1][column], kde=False, norm_hist=True,
            bins=5,color='b', label='성공')
sns.distplot(x_test[y_test==0][column], kde=False, norm_hist=True,
             bins=5,color='k', label='실패')

plt.legend()
plt.show()

In [ ]:
column = '주택대출'

sns.distplot(x_test[y_test==1][column], kde=False, norm_hist=True,
            bins=5,color='b', label='성공')
sns.distplot(x_test[y_test==0][column], kde=False, norm_hist=True,
             bins=5,color='k', label='실패')

plt.legend()
plt.show()

In [ ]:
column = '혼인_여부_single'

sns.distplot(x_test[y_test==1][column], kde=False, norm_hist=True,
            bins=5,color='b', label='성공')
sns.distplot(x_test[y_test==0][column], kde=False, norm_hist=True,
             bins=5,color='k', label='실패')

plt.legend()
plt.show()